# 一、环境check

In [6]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

1.13.1+cu117 True


In [3]:
# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

1.2.0


In [4]:
# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

11.8
GCC 7.5


In [5]:
# Check MMEngine installation
from mmengine.utils.dl_utils import collect_env
print(collect_env())

OrderedDict([('sys.platform', 'linux'), ('Python', '3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'), ('CUDA available', True), ('MUSA available', False), ('numpy_random_seed', 2147483648), ('GPU 0,1,2,3,4,5,6,7', 'NVIDIA A100-PCIE-40GB'), ('CUDA_HOME', '/usr/local/cuda'), ('NVCC', 'Cuda compilation tools, release 11.8, V11.8.89'), ('GCC', 'gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0'), ('PyTorch', '1.13.1+cu117'), ('PyTorch compiling details', 'PyTorch built with:\n  - GCC 9.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - LAPACK is enabled (usually provided by MKL)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.7\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,co

# 二、使用MMAction2识别器执行推理
MMAction2已经提供了高级API来进行推理和训练。

In [8]:
# TSN-R50-1x1x3 作为基于 rgb 的动作识别器
# checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth

In [10]:
from mmaction.apis import inference_recognizer, init_recognizer
from mmengine import Config


# Choose to use a config and initialize the recognizer
config = '../configs/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py'
config = Config.fromfile(config)
# Setup a checkpoint file to load
checkpoint = '../checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'
# Initialize the recognizer
model = init_recognizer(config, checkpoint, device='cuda:0')

Loads checkpoint by local backend from path: ../checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth


In [11]:
# Use the recognizer to do inference
from operator import itemgetter
video = '../demo/demo.mp4'
label = '../tools/data/kinetics/label_map_k400.txt'
results = inference_recognizer(model, video)

pred_scores = results.pred_score.tolist()
score_tuples = tuple(zip(range(len(pred_scores)), pred_scores))
score_sorted = sorted(score_tuples, key=itemgetter(1), reverse=True)
top5_label = score_sorted[:5]

labels = open(label).readlines()
labels = [x.strip() for x in labels]
results = [(labels[k[0]], k[1]) for k in top5_label]


10/17 02:56:03 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
10/17 02:56:03 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.


In [12]:
print('The top-5 labels with corresponding scores are:')
for result in results:
    print(f'{result[0]}: ', result[1])

The top-5 labels with corresponding scores are:
arm wrestling:  1.0
rock scissors paper:  6.434453414527752e-09
shaking hands:  2.758343997655288e-09
clapping:  1.3451096902983295e-09
massaging feet:  5.551171189388526e-10


# 三、在自定义数据集上训练识别器
要训练新的识别器，通常有三件事要做：

    支持新数据集
    修改配置
    训练新的识别器

## 支持新数据集
在本教程中，我们给出了一个将数据转换为现有数据集格式的示例。其他方法和更高级的用法可以在文档中找到
首先，让我们下载一个从Kinetics-400获得的小数据集。我们选择了30个带有标签的视频作为训练数据集，10个带有标签视频作为测试数据集。

## 修改配置
下一步，我们需要修改训练的配置。为了加速这一过程，我们使用预训练的识别器对识别器进行微调

TSN 模型中的帧采样策略 1x1x3 表示什么？

在 TSN（Temporal Segment Networks） 模型中，帧采样策略通常以 N x L x T 的形式表示，其中：

ampleFrames 定义输入帧的示例策略。示例策略定义为clip_len xframe_interval x num_clips。

    clip_len=1意味着每个剪辑只包含1帧。
    frame_interval=1意味着每帧都被选取。
    num_clips=3意味着从视频中会抽取3个剪辑。
密集策略演示：32x2x1可选择1、3、5、...、63帧。

统一策略演示：1x1x8 将 100 帧分为 [1, 12]、[13, 24]、[25, 36]、[37, 48]、...、[85, 96]，并且可以选择 1、13 , 25, ..., 85 帧。及将视频分为8个片段，每frame_interval 1 帧采样一帧，最终每个片段得到clip_len帧。



In [25]:
cfg = Config.fromfile('../configs/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py')

给定一个在kinetics400完整数据集上训练TSN模型的配置，我们需要修改一些值，将其用于在kinetics400微小数据集上培训TSN。

In [28]:
from mmengine.runner import set_random_seed

# Modify dataset type and path
cfg.data_root = '/root/project/research/action/mmaction2/data/kinetics400_tiny/train/'
cfg.data_root_val = '/root/project/research/action/mmaction2/data/kinetics400_tiny/val/'
cfg.ann_file_train = '/root/project/research/action/mmaction2/data/kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.ann_file_val = '/root/project/research/action/mmaction2/data/kinetics400_tiny/kinetics_tiny_val_video.txt'


cfg.test_dataloader.dataset.ann_file = '/root/project/research/action/mmaction2/data/kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.test_dataloader.dataset.data_prefix.video = '/root/project/research/action/mmaction2/data/kinetics400_tiny/val/'

cfg.train_dataloader.dataset.ann_file = '/root/project/research/action/mmaction2/data/kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.train_dataloader.dataset.data_prefix.video = '/root/project/research/action/mmaction2/data/kinetics400_tiny/train/'

cfg.val_dataloader.dataset.ann_file = '/root/project/research/action/mmaction2/data/kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.val_dataloader.dataset.data_prefix.video  = '/root/project/research/action/mmaction2/data/kinetics400_tiny/val/'


# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 2
# We can use the pre-trained TSN model
cfg.load_from = '/root/project/research/action/mmaction2/checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
# 原本的配置是为 8-GPU 的训练设置的。通常，多GPU训练时会采用较大的 batch size（每个 GPU 上有一定的 batch size，总的 batch size 为多个 GPU 的总和）。
print(f"batch size: {cfg.train_dataloader.batch_size}")
cfg.train_dataloader.batch_size = cfg.train_dataloader.batch_size // 16
cfg.val_dataloader.batch_size = cfg.val_dataloader.batch_size // 16
print(f"batch_size : {cfg.train_dataloader.batch_size}, {cfg.val_dataloader.batch_size}")
# 学习率（LR） 也需要根据 GPU 数量 和 batch size 进行调整。
# 原始学习率是为 8-GPU 训练设计的。常见的经验法则是：学习率与总的 batch size 成比例。当 batch size 减少时，学习率也应该相应减少。
# 这里，学习率先除以 8，是为了适应从 8 个 GPU 到 1 个 GPU 的转换，再除以 16，是因为 batch size 也相应缩小了 16 倍。
cfg.optim_wrapper.optimizer.lr = cfg.optim_wrapper.optimizer.lr / 8 / 16
# 小数据集收敛速度更快，过多的 epochs 可能导致 过拟合。
cfg.train_cfg.max_epochs = 10

# num_workers 指定了数据加载过程中用于数据预处理的工作线程数。更多的工作线程可以加速数据加载，尤其是对于大数据集。
# 但是对于 小数据集（如 kinetics400_tiny），较少的工作线程已经足够完成数据加载，且在单GPU训练时，减少 num_workers 也有助于节省系统资源。
cfg.train_dataloader.num_workers = 2
cfg.val_dataloader.num_workers = 2
cfg.test_dataloader.num_workers = 2

# We can initialize the logger for training and have a look at the final config used for training
# 我们可以初始化记录器进行训练，并查看用于训练的最终配置
print(f'Config:\n{cfg.pretty_text}')


batch size: 2
batch_size : 0, 0
Config:
ann_file_train = '/root/project/research/action/mmaction2/data/kinetics400_tiny/kinetics_tiny_train_video.txt'
ann_file_val = '/root/project/research/action/mmaction2/data/kinetics400_tiny/kinetics_tiny_val_video.txt'
auto_scale_lr = dict(base_batch_size=256, enable=False)
data_root = '/root/project/research/action/mmaction2/data/kinetics400_tiny/train/'
data_root_val = '/root/project/research/action/mmaction2/data/kinetics400_tiny/val/'
dataset_type = 'VideoDataset'
default_hooks = dict(
    checkpoint=dict(
        interval=3, max_keep_ckpts=3, save_best='auto', type='CheckpointHook'),
    logger=dict(ignore_last=False, interval=20, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    runtime_info=dict(type='RuntimeInfoHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    sync_buffers=dict(type='SyncBuffersHook'),
    timer=dict(type='IterTimerHook'))
default_scope = 'mmaction'
env_cfg = dict(
    cudnn_benchmar

# 四、训练新的识别器
最后，让我们初始化数据集和识别器，然后训练一个新的识别器！

In [27]:
import os.path as osp
import mmengine
from mmengine.runner import Runner

# Create work_dir
mmengine.mkdir_or_exist(osp.abspath(cfg.work_dir))

# build the runner from config
runner = Runner.from_cfg(cfg)

# start training
runner.train()

10/17 08:38:29 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1605309348
    GPU 0,1,2,3,4,5,6,7: NVIDIA A100-PCIE-40GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
    PyTorch: 1.13.1+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


10/17 08:49:35 - mmengine - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.weight', 'fc.bias'}
Loads checkpoint by local backend from path: /root/project/research/action/mmaction2/checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 2048]) from checkpoint, the shape in current model is torch.Size([2, 2048]).
size mismatch for cls_head.fc_cls.bias: copying a param with shape torch.Size([400]) from checkpoint, the shape in current model is torch.Size([2]).
10/17 08:49:36 - mmengine - INFO - Load checkpoint from /root/project/research/action/mmaction2/checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
10/17 08:49:36 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
10/17 08:49:36 - mmengi

Recognizer2D(
  (data_preprocessor): ActionDataPreprocessor()
  (backbone): ResNet(
    (conv1): ConvModule(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): ConvModule(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=True)
        )
        (conv2): ConvModule(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=True)
        

# 五、了解日志
从日志中，我们可以对训练过程有一个基本的了解，并知道识别器的训练情况。

首先，加载在ImageNet上预训练的ResNet-50骨干网，这是一种常见的做法，因为从头开始训练成本更高。日志显示，除了fc.bias和fc.weight之外，ResNet-50骨干网的所有权重都已加载。

其次，由于我们使用的数据集很小，我们加载了一个TSN模型并对其进行微调以进行动作识别。原始TSN是在原始Kinetics-400数据集上训练的，该数据集包含400个类，但Kinetics-400Tiny数据集只有2个类。因此，用于分类的预训练TSN的最后一个FC层具有不同的权重形状，因此未被使用。

第三，训练后，识别器将按默认评估进行评估。结果表明，该识别器在val数据集上实现了100%的top1准确率和100%的top5准确率，

不错！

# 六、测试训练好的识别器
在对识别器进行微调之后，让我们检查一下预测结果！

In [29]:
runner.test()

10/17 08:53:35 - mmengine - INFO - Epoch(test) [10/10]    acc/top1: 0.9000  acc/top5: 1.0000  acc/mean1: 0.9000  data_time: 0.0673  time: 0.2694


{'acc/top1': 0.9, 'acc/top5': 1.0, 'acc/mean1': 0.9}

# 七、模型结构

## TSN模型
该模型被定义为一个2D动作识别器，主要用于处理视频数据并识别其中的动作。

结构通过结合ResNet50作为特征提取器和TSNHead作为分类器，能够有效处理视频数据并识别多个动作类别。

### backbone
backbone部分是模型的主干网络，负责提取视频帧中的特征。 选择resnet50

### cls_head
cls_head部分是模型的分类头，负责将提取的特征映射到动作类别。

type='TSNHead'：指定分类头的类型为TSNHead，适用于TSN模型的分类任务。

    平均池化层
    共识层
    Dropout 层
    线性分类层 fc_cls
consensus=dict(type='AvgConsensus', dim=1)：定义共识函数为平均共识，沿着维度1进行操作，结合多个帧的特征以获得最终的分类结果。

    AvgConsensus 是一个用于平均共识的模块，通常在处理视频分类任务时用于融合不同时间段的特征。

    理解参考：https://blog.csdn.net/irving512/article/details/107501281
    
    输入: [N, num_segs, in_channels, 1, 1]， 输入的特征图尺寸其实是 N * num_segs，即包括了 batch size 以及一个clip中的T帧图片。在1x1x3中num_segs==1
    每个样本的 num_segs 个时间段都有其对应的特征表示。 AvgConsensus 会计算时间维度上的平均值。
    输出: [N, 1, in_channels, 1, 1]。这使得后续的分类头（如 TSNHead）可以更方便地处理这些融合后的特征。
    
    经过共识操作后，得到的特征只保留了一次样本的平均表示，去掉了 num_segs 维度。这个特征是对整个视频在所有时间段上进行融合的结果。
